# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

# 고객별 백화점 구매기록 데이터셋
- 미혼(1) or 기혼(0) 고객을 예측
- 학습용 구매기록 데이터
    - https://drive.google.com/file/d/1tg41qXu02FK55bFa8P1Gx2URhwPUCoQQ/view?usp=sharing
- 학습용 정답 데이터
    - https://drive.google.com/file/d/1-9AcU9nAoO4SzSmqdCOYTvWNpA5Pdt3q/view?usp=sharing
- 테스트용 구매기록 데이터
    - https://drive.google.com/file/d/1-AwDfGlHm9rNtpnHIWOK96jBJYy3f2SZ/view?usp=sharing
- 제출 양식 데이터
    - https://drive.google.com/file/d/1-Qv7SlsY5Eu3bRR7Z0IUJyLkOO1Fl3y6/view?usp=sharing

In [554]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [555]:
# DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
# DATA_PATH

In [556]:
DATA_PATH = "data/"
DATA_PATH

'data/'

- 데이터 불러오기

In [636]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [558]:
from IPython.display import display, HTML
br_html = HTML("<br><br>")
display(HTML("<h1>학습용 구매기록 데이터</h1>"), train_tr.head(), br_html)
display(HTML("<h1>학습용 정답 데이터</h1>"), train_target.head(), br_html)
display(HTML("<h1>테스트용 구매기록 데이터</h1>"), test_tr.head(), br_html)
display(HTML("<h1>제출 양식 데이터</h1>"), submit.head(), br_html)

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000


,ID,target
0,train_0,1.0
1,train_1,1.0
2,train_2,0.0
3,train_3,0.0
4,train_4,0.0


,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
0,test_3366,2004-05-01 10:20:00,A373000,생식품,건식품,5100,7000
1,test_9389,2004-05-01 10:23:00,A373000,명품잡화,스타킹,5104,46000
2,test_8190,2004-05-01 10:30:00,A144000,잡화파트,수입종합화장품,5106,260000
3,test_8835,2004-05-01 10:30:00,A144000,잡화파트,준보석,5107,17000
4,test_1167,2004-05-01 10:30:00,A373000,남성의류,셔츠,5108,138000


,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5


In [559]:
display(HTML("<h1>학습용 구매기록 데이터</h1>"))
display(train_tr.info(), br_html)

display(HTML("<h1>학습용 정답 데이터</h1>"))
display(train_target.info(), br_html)

display(HTML("<h1>테스트용 구매기록 데이터</h1>"))
display(test_tr.info(), br_html)

display(HTML("<h1>제출 양식 데이터</h1>"))
display(submit.info(), br_html)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523105 entries, 0 to 523104
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      523105 non-null  object
 1   구매일시    523105 non-null  object
 2   지점코드    523105 non-null  object
 3   대분류     523105 non-null  object
 4   중분류     523105 non-null  object
 5   브랜드코드   523105 non-null  int64 
 6   구매가격    523105 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 27.9+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14940 entries, 0 to 14939
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      14940 non-null  object 
 1   target  14940 non-null  float64
dtypes: float64(1), object(1)
memory usage: 233.6+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441196 entries, 0 to 441195
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      441196 non-null  object
 1   구매일시    441196 non-null  object
 2   지점코드    441196 non-null  object
 3   대분류     441196 non-null  object
 4   중분류     441196 non-null  object
 5   브랜드코드   441196 non-null  int64 
 6   구매가격    441196 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 23.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12225 entries, 0 to 12224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      12225 non-null  object 
 1   target  12225 non-null  float64
dtypes: float64(1), object(1)
memory usage: 191.1+ KB


None

In [560]:
train_tr.describe()

,브랜드코드,구매가격
count,523105.000000,5.231050e+05
mean,5391.568913,9.873469e+04
std,380.975185,3.134026e+05
min,5100.000000,-3.082000e+07
25%,5111.000000,2.252000e+04
50%,5237.000000,5.200000e+04
75%,5522.000000,1.197000e+05
max,6996.000000,3.082000e+07


# 특성 공학(Feature Engineering)

## 날짜 형식으로 변환

In [637]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

## 날짜, 시간 컬럼 분할

In [638]:
train_tr["구매날짜"] = train_tr["구매일시"].dt.date
test_tr["구매날짜"] = test_tr["구매일시"].dt.date

train_tr["구매시각"] = train_tr["구매일시"].dt.time
test_tr["구매시각"] = test_tr["구매일시"].dt.time

train_tr = train_tr.drop(columns=["구매일시"])
test_tr = test_tr.drop(columns=["구매일시"])

train_tr["구매날짜"] = pd.to_datetime(train_tr["구매날짜"], errors="coerce")
test_tr["구매날짜"] = pd.to_datetime(test_tr["구매날짜"], errors="coerce")

train_tr.shape, test_tr.shape

((523105, 8), (441196, 8))

## 환불한 내역에 대한 구매내역 및 환불내역 삭제

In [563]:
refund_train = train_tr[train_tr["구매가격"] < 0].reset_index()
refund_test = test_tr[test_tr["구매가격"] < 0].reset_index()

In [641]:
# 환불 금액의 절대값과 동일한 구매 내역 찾기
refund_pairs_train = pd.merge(
    refund_train,
    train_tr[train_tr["구매가격"] > 0].reset_index(),
    on=["ID", "지점코드", "대분류", "중분류", "브랜드코드"],
    suffixes=("_refund", "_purchase")
)

# 환불 금액이 일치하는 경우만 선택
# refund_pairs_train = refund_pairs_train[
#     refund_pairs_train["구매가격_refund"].abs() == refund_pairs_train["구매가격_purchase"]
# ].copy()

# 환불 날짜 이후에 환불 금액과 일치하는 구매건
refund_pairs_train = refund_pairs_train[
    refund_pairs_train["구매날짜_purchase"] > refund_pairs_train["구매날짜_refund"]
]

repurchase_count = refund_pairs_train.groupby("ID").size().reset_index(name='환불후재구매브랜드_count')

repurchase_count

,ID,환불후재구매브랜드_count
0,train_0,1
1,train_10,2
2,train_100,1
3,train_1000,4
4,train_10000,1
...,...,...
3338,train_9983,9
3339,train_9986,3
3340,train_9988,1
3341,train_9997,1


In [633]:
# 테스트 데이터
refund_pairs_test = pd.merge(
    refund_test,
    test_tr[test_tr["구매가격"] > 0].reset_index(),
    on=["ID", "지점코드", "대분류", "중분류", "브랜드코드"],
    suffixes=("_refund", "_purchase")
)

refund_pairs_test = refund_pairs_test[
    refund_pairs_test["구매가격_refund"].abs() == refund_pairs_test["구매가격_purchase"]
].copy()

refund_pairs_test = refund_pairs_test[
    refund_pairs_test["구매날짜_purchase"] > refund_pairs_test["구매날짜_refund"]
]

repurchase_count_test = refund_pairs_test.groupby("ID").size().reset_index(name='환불후재구매_count')

repurchase_count_test

,ID,환불후재구매_count
0,test_10004,1
1,test_10019,2
2,test_10042,3
3,test_1005,1
4,test_10074,1
...,...,...
429,test_9932,1
430,test_9940,1
431,test_995,1
432,test_9974,2


In [564]:
# 환불 금액의 절대값과 동일한 구매 내역 찾기
refund_pairs_train = pd.merge(
    refund_train,
    train_tr[train_tr["구매가격"] > 0].reset_index(),
    on=["ID", "지점코드", "대분류", "중분류", "브랜드코드"],
    suffixes=("_refund", "_purchase")
)

# 환불 금액이 일치하는 경우만 선택
refund_pairs_train = refund_pairs_train[
    refund_pairs_train["구매가격_refund"].abs() == refund_pairs_train["구매가격_purchase"]
].copy()

# 각 환불건에 대해 하나의 구매건만 매칭되도록 선택
refund_pairs_train = refund_pairs_train.groupby(
    ["ID", "구매가격_refund", "지점코드", "대분류", "중분류", "브랜드코드"]
).first().reset_index()

# 테스트 데이터에도 동일하게 적용
refund_pairs_test = pd.merge(
    refund_test,
    test_tr[test_tr["구매가격"] > 0].reset_index(),
    on=["ID", "지점코드", "대분류", "중분류", "브랜드코드"],
    suffixes=("_refund", "_purchase")
)

refund_pairs_test = refund_pairs_test[
    refund_pairs_test["구매가격_refund"].abs() == refund_pairs_test["구매가격_purchase"]
].copy()

refund_pairs_test = refund_pairs_test.groupby(
    ["ID", "구매가격_refund", "지점코드", "대분류", "중분류", "브랜드코드"]
).first().reset_index()

# 환불 내역과 해당 구매 내역의 인덱스 찾기
train_refund_indices = refund_pairs_train['index_refund'].values
train_purchase_indices = refund_pairs_train['index_purchase'].values

test_refund_indices = refund_pairs_test['index_refund'].values
test_purchase_indices = refund_pairs_test['index_purchase'].values

# 환불 내역과 해당 구매 내역 모두 제거
train_tr_clean = train_tr[~train_tr.index.isin(train_refund_indices) & 
                         ~train_tr.index.isin(train_purchase_indices)].copy()
test_tr_clean = test_tr[~test_tr.index.isin(test_refund_indices) & 
                       ~test_tr.index.isin(test_purchase_indices)].copy()

# 인덱스 초기화
train_tr_clean = train_tr_clean.reset_index(drop=True)
test_tr_clean = test_tr_clean.reset_index(drop=True)

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [565]:
train_ft = train_target[["ID"]]
test_ft = submit[["ID"]]

train_ft.shape, test_ft.shape

((14940, 1), (12225, 1))

## 구매날짜를 이용한 특성생성

In [566]:
# train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
# test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

In [567]:
# 성수기/비수기 구매비율 추가
# 분기별 구매비율 추가
# 백화점이용기간, 구매간격 표준편차 추가
# 월초, 월중순, 월말 구매비율, 마지막구매후 경과일 추가(2005-04-30 기준)

agg_list = [
    # 일
    ("총방문일수", "nunique"),
    ("첫구매날짜", 'min'),
    ("마지막구매날짜", 'max'),
    ("백화점이용기간", lambda x: (x.max() - x.min()).days),
    ("구매주기", lambda x: int((x.max() - x.min()).days / x.dt.date.nunique())),
    ("주말방문일수", lambda x: x[x.dt.weekday > 4].nunique()),
    ("일별평균구매건수", lambda x: x.count() / x.dt.date.nunique()),
    ("구매간격_표준편차", lambda x: np.std(np.diff([d.toordinal() for d in sorted(x)])) if len(x) > 1 else 0),
    ("마지막구매후_경과일", lambda x: (pd.Timestamp('2005-04-30') - x.max()).days),
    
    # 요일
    ('평균구매요일', lambda x: x.dt.weekday.mean()),
    ("주구매요일", lambda x: x.dt.weekday.mode()[0]),
    ("월요일_구매비율", lambda x: np.mean(x.dt.weekday == 0)),
    ("화요일_구매비율", lambda x: np.mean(x.dt.weekday == 1)),
    ("수요일_구매비율", lambda x: np.mean(x.dt.weekday == 2)),
    ("목요일_구매비율", lambda x: np.mean(x.dt.weekday == 3)),
    ("금요일_구매비율", lambda x: np.mean(x.dt.weekday == 4)),
    ("토요일_구매비율", lambda x: np.mean(x.dt.weekday == 5)),
    ("일요일_구매비율", lambda x: np.mean(x.dt.weekday == 6)),
    
    # 월
    ("1월_구매비율", lambda x: np.mean(x.dt.month == 1)),
    ("2월_구매비율", lambda x: np.mean(x.dt.month == 2)),
    ("3월_구매비율", lambda x: np.mean(x.dt.month == 3)),
    ("4월_구매비율", lambda x: np.mean(x.dt.month == 4)),
    ("5월_구매비율", lambda x: np.mean(x.dt.month == 5)),
    ("6월_구매비율", lambda x: np.mean(x.dt.month == 6)),
    ("7월_구매비율", lambda x: np.mean(x.dt.month == 7)),
    ("8월_구매비율", lambda x: np.mean(x.dt.month == 8)),
    ("9월_구매비율", lambda x: np.mean(x.dt.month == 9)),
    ("10월_구매비율", lambda x: np.mean(x.dt.month == 10)),
    ("11월_구매비율", lambda x: np.mean(x.dt.month == 11)),
    ("12월_구매비율", lambda x: np.mean(x.dt.month == 12)),
    ("거래개월수", lambda x: x.dt.date.astype(str).str[:-3].nunique()),
    ("월별평균구매건수", lambda x: x.count() / x.dt.month.nunique()),
    ("월초구매비율", lambda x: np.mean(x.dt.day <= 10)),
    ("월중순구매비율", lambda x: np.mean((x.dt.day > 10) & (x.dt.day <= 20))),
    ("월말구매비율", lambda x: np.mean(x.dt.day >= 21)),
    
    ("구매횟수_상반기", lambda x: np.mean((x.dt.month >= 1) & (x.dt.month <= 6))),
    ("구매횟수_하반기", lambda x: np.mean((x.dt.month >= 7) & (x.dt.month <= 12))),
    
    # 성수기
    ("추석성수기_구매비율", lambda x: np.mean((x.dt.month == 9) & (x.dt.day >= 18) | (x.dt.month == 10) & (x.dt.day <= 3))),
    ("연말연시_구매비율", lambda x: np.mean((x.dt.month == 12) | (x.dt.month == 1) & (x.dt.day <= 10))),
    ("설날성수기_구매비율", lambda x: np.mean((x.dt.month == 2) & (x.dt.day >= 1) & (x.dt.day <= 15))),
    
    # 준성수기
    ("여름시즌_구매비율", lambda x: np.mean((x.dt.month == 6) & (x.dt.day >= 15) | (x.dt.month == 7) | (x.dt.month == 8) & (x.dt.day <= 15))),
    ("신학기_구매비율", lambda x: np.mean((x.dt.month == 3) & (x.dt.day >= 1) & (x.dt.day <= 15))),
    
    # 중간기
    ("여름끝추석전_구매비율", lambda x: np.mean((x.dt.month == 8) & (x.dt.day >= 16) | (x.dt.month == 9) & (x.dt.day <= 17))),
    ("연말연시끝설날전_구매비율", lambda x: np.mean((x.dt.month == 1) & (x.dt.day >= 11) & (x.dt.day <= 31))),
    ("신학기후_구매비율", lambda x: np.mean((x.dt.month == 3) & (x.dt.day >= 16) | (x.dt.month == 4) & (x.dt.day <= 10))),
    
    # 비수기
    ("초여름_구매비율", lambda x: np.mean((x.dt.month == 5) & (x.dt.day >= 17) | (x.dt.month == 6) & (x.dt.day <= 14))),
    ("추석후연말전_구매비율", lambda x: np.mean((x.dt.month == 10) & (x.dt.day >= 4) | (x.dt.month == 11))),
    ("봄철비수기_구매비율", lambda x: np.mean((x.dt.month == 4) & (x.dt.day >= 11) & (x.dt.day <= 29))),
    
    # 계절
    ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
    ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
    ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
    
    #분기
    ("1분기_구매비율", lambda x: np.mean(x.dt.quarter == 1)),
    ("2분기_구매비율", lambda x: np.mean(x.dt.quarter == 2)),
    ("3분기_구매비율", lambda x: np.mean(x.dt.quarter == 3)),
    ("4분기_구매비율", lambda x: np.mean(x.dt.quarter == 4)),
    
    # 년
    ("2004년_구매비율", lambda x: np.mean(x.dt.year == 2004)),
    ("2005년_구매비율", lambda x: np.mean(x.dt.year == 2005)),
    ("연도별평균구매건수", lambda x: x.count() / x.dt.year.nunique()),
]

tmp = train_tr_clean.groupby("ID")["구매날짜"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how="left", on="ID")
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,여름_구매비율,가을_구매비율,겨울_구매비율,1분기_구매비율,2분기_구매비율,3분기_구매비율,4분기_구매비율,2004년_구매비율,2005년_구매비율,연도별평균구매건수
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,0.357143,0.285714,0.285714,0.071429,0.285714,0.428571,0.214286,0.928571,0.071429,7.0
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,0.166667,0.357143,0.119048,0.190476,0.238095,0.380952,0.190476,0.761905,0.238095,21.0
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,0.120000,0.200000,0.230000,0.310000,0.400000,0.150000,0.140000,0.520000,0.480000,50.0
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,0.194595,0.194595,0.221622,0.275676,0.340541,0.162162,0.221622,0.551351,0.448649,92.5
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,0.472222,0.305556,0.111111,0.166667,0.194444,0.527778,0.111111,0.833333,0.166667,18.0


In [568]:
tmp = test_tr_clean.groupby("ID")["구매날짜"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how="left", on="ID")
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,여름_구매비율,가을_구매비율,겨울_구매비율,1분기_구매비율,2분기_구매비율,3분기_구매비율,4분기_구매비율,2004년_구매비율,2005년_구매비율,연도별평균구매건수
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,0.285714,0.428571,0.000000,0.000000,0.428571,0.142857,0.428571,1.000000,0.000000,7.0
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,0.000000,0.750000,0.250000,0.250000,0.000000,0.250000,0.500000,0.750000,0.250000,2.0
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,0.600000,0.400000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,5.0
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,0.458333,0.375000,0.083333,0.041667,0.083333,0.583333,0.291667,0.875000,0.125000,12.0
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,0.266667,0.333333,0.133333,0.133333,0.266667,0.266667,0.333333,0.866667,0.133333,7.5


In [569]:
train_ft.to_csv(f"{DATA_PATH}train_tmp.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_tmp.csv",index=False)

## 구매시각을 이용한 특성생성

In [570]:
train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

In [571]:
# train_tr['요일'] = train_tr['구매날짜'].dt.weekday
# train_tr['평일주말'] = train_tr['요일'].apply(lambda x: '주말' if x >= 5 else '평일')
# train_tr['시간대'] = train_tr['구매시각'].apply(lambda x: '18시 이후' if int(str(x)[:2]) >= 18 else '18시 이전')

# train_tmp = train_tr.pivot_table(
#     index='ID',
#     columns=['평일주말', '시간대'],
#     values='구매가격',
#     aggfunc='count',
#     fill_value=0
# )

# train_tmp.columns = [f"{col[0]}_{col[1]}_구매횟수" for col in train_tmp.columns]
# train_tmp = train_tmp.reset_index()

# train_ft = train_ft.merge(train_tmp, 'left', 'ID')
# train_ft.head()

In [572]:
# test_tr['요일'] = test_tr['구매날짜'].dt.weekday
# test_tr['평일주말'] = test_tr['요일'].apply(lambda x: '주말' if x >= 5 else '평일')
# test_tr['시간대'] = test_tr['구매시각'].apply(lambda x: '18시 이후' if int(str(x)[:2]) >= 18 else '18시 이전')

# test_tmp = test_tr.pivot_table(
#     index='ID',
#     columns=['평일주말', '시간대'],
#     values='구매가격',
#     aggfunc='count',
#     fill_value=0
# )

# test_tmp.columns = [f"{col[0]}_{col[1]}_구매횟수" for col in test_tmp.columns]
# test_tmp = test_tmp.reset_index()

# test_ft = test_ft.merge(test_tmp, 'left', 'ID')
# test_ft.head()

- 구매시간대 컬럼생성

In [573]:
def shopping_time(df):
    # 날짜별 쇼핑시간 계산
    shopping_by_date = df.groupby(['ID', '구매날짜']).agg({
        '구매시각': lambda x: (max(x).hour * 60 + max(x).minute - (min(x).hour * 60 + min(x).minute)) / 60
    }).reset_index()
    shopping_by_date.columns = ['ID', '구매날짜', '쇼핑시간']
    
    # 쇼핑시간이 있는 경우만 통계 계산
    shopping_stats = shopping_by_date[shopping_by_date['쇼핑시간'] > 0].groupby('ID').agg({
        '쇼핑시간': ['mean', 'std', 'max', 'min', 'count']
    }).reset_index()
    
    shopping_stats.columns = [
        'ID', 
        '평균쇼핑시간', 
        '쇼핑시간_표준편차', 
        '최대쇼핑시간', 
        '최소쇼핑시간',
        '쇼핑일수'
    ]
    
    return shopping_stats

# train/test 적용
train_shopping = shopping_time(train_tr_clean)

# 기존 feature에 병합
train_ft = train_ft.merge(train_shopping, on='ID', how='left')

# NaN값 처리
shopping_cols = ['평균쇼핑시간', '쇼핑시간_표준편차', '최대쇼핑시간', '최소쇼핑시간', '쇼핑일수']
train_ft[shopping_cols] = train_ft[shopping_cols].fillna(0)
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,3분기_구매비율,4분기_구매비율,2004년_구매비율,2005년_구매비율,연도별평균구매건수,평균쇼핑시간,쇼핑시간_표준편차,최대쇼핑시간,최소쇼핑시간,쇼핑일수
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,0.428571,0.214286,0.928571,0.071429,7.0,0.372222,0.227507,0.616667,0.166667,3.0
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,0.380952,0.190476,0.761905,0.238095,21.0,1.075758,1.085059,4.050000,0.100000,11.0
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,0.150000,0.140000,0.520000,0.480000,50.0,0.874074,1.183481,4.633333,0.033333,27.0
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,0.162162,0.221622,0.551351,0.448649,92.5,1.822868,2.113224,7.166667,0.033333,43.0
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,0.527778,0.111111,0.833333,0.166667,18.0,0.416667,0.091287,0.500000,0.333333,6.0


In [574]:
test_shopping = shopping_time(test_tr_clean)
test_ft = test_ft.merge(test_shopping, on='ID', how='left')
test_ft[shopping_cols] = test_ft[shopping_cols].fillna(0)
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,3분기_구매비율,4분기_구매비율,2004년_구매비율,2005년_구매비율,연도별평균구매건수,평균쇼핑시간,쇼핑시간_표준편차,최대쇼핑시간,최소쇼핑시간,쇼핑일수
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,0.142857,0.428571,1.000000,0.000000,7.0,0.750000,0.589256,1.166667,0.333333,2.0
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,0.250000,0.500000,0.750000,0.250000,2.0,0.166667,0.000000,0.166667,0.166667,1.0
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,1.000000,0.000000,1.000000,0.000000,5.0,0.166667,0.000000,0.166667,0.166667,2.0
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,0.583333,0.291667,0.875000,0.125000,12.0,0.733333,0.550757,1.300000,0.200000,3.0
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,0.266667,0.333333,0.866667,0.133333,7.5,1.244444,0.719825,2.000000,0.566667,3.0


In [575]:
# 시간 간격 수정

agg_list = [
    ("18시이전_구매비율", lambda x: np.mean([t.hour < 18 for t in x])),
    ("18시이후_구매비율", lambda x: np.mean([t.hour >= 18 for t in x])),
    ("오전_구매비율", lambda x: np.mean([t.hour < 12 for t in x])),
    ("오후_구매비율", lambda x: np.mean([t.hour >= 12 for t in x])),
    ("주구매시간대", lambda x: pd.Series([t.hour for t in x]).mode()[0]),
    ("평균구매시간", lambda x: np.mean([t.hour + t.minute/60 for t in x])),
    ("야간구매비율", lambda x: np.mean([(t.hour >= 18) | (t.hour <= 6) for t in x])),
    ("새벽구매비율", lambda x: np.mean([(t.hour >= 0) & (t.hour < 6) for t in x])),
    ("아침_구매비율", lambda x: np.mean([6 <= t.hour < 11 for t in x])),
    ("점심_구매비율", lambda x: np.mean([11 <= t.hour < 14 for t in x])),
    ("저녁_구매비율", lambda x: np.mean([17 <= t.hour < 21 for t in x])),
]

tmp = train_tr_clean.groupby("ID")["구매시각"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how="left", on="ID")
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,18시이후_구매비율,오전_구매비율,오후_구매비율,주구매시간대,평균구매시간,야간구매비율,새벽구매비율,아침_구매비율,점심_구매비율,저녁_구매비율
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,0.357143,0.000000,1.000000,12,16.125000,0.357143,0.0,0.000000,0.357143,0.571429
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,0.309524,0.047619,0.952381,18,15.615873,0.309524,0.0,0.023810,0.380952,0.428571
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,0.150000,0.070000,0.930000,16,15.570500,0.150000,0.0,0.020000,0.200000,0.260000
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,0.221622,0.064865,0.935135,17,15.731802,0.221622,0.0,0.010811,0.248649,0.378378
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,0.222222,0.111111,0.888889,16,15.694444,0.222222,0.0,0.027778,0.222222,0.333333


In [576]:
tmp = test_tr_clean.groupby("ID")["구매시각"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how="left", on="ID")
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,18시이후_구매비율,오전_구매비율,오후_구매비율,주구매시간대,평균구매시간,야간구매비율,새벽구매비율,아침_구매비율,점심_구매비율,저녁_구매비율
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,0.000000,0.00,1.00,14,14.078571,0.000000,0.0,0.0,0.428571,0.000000
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,0.000000,0.25,0.75,11,15.095833,0.000000,0.0,0.0,0.250000,0.250000
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,0.000000,0.40,0.60,17,15.066667,0.000000,0.0,0.0,0.400000,0.600000
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,0.541667,0.00,1.00,18,17.884722,0.541667,0.0,0.0,0.000000,0.708333
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,0.466667,0.00,1.00,19,17.278889,0.466667,0.0,0.0,0.066667,0.600000


In [577]:
train_ft.to_csv(f"{DATA_PATH}train_tmp.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_tmp.csv",index=False)

## 지점을 이용한 특성생성

In [578]:
train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

In [579]:
agg_list = [
    ("방문지점수", "nunique"),
    ("주구매지점", lambda x: x.mode()[0]),
    ("주구매지점_이용비율", lambda x: x[x == x.mode()[0]].count() / x.count()),
]

tmp = train_tr_clean.groupby("ID")["지점코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how="left", on="ID")
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,주구매시간대,평균구매시간,야간구매비율,새벽구매비율,아침_구매비율,점심_구매비율,저녁_구매비율,방문지점수,주구매지점,주구매지점_이용비율
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,12,16.125000,0.357143,0.0,0.000000,0.357143,0.571429,2,A112000,0.571429
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,18,15.615873,0.309524,0.0,0.023810,0.380952,0.428571,3,A112000,0.738095
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,16,15.570500,0.150000,0.0,0.020000,0.200000,0.260000,2,A373000,0.900000
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,17,15.731802,0.221622,0.0,0.010811,0.248649,0.378378,3,A144000,0.805405
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,16,15.694444,0.222222,0.0,0.027778,0.222222,0.333333,2,A144000,0.888889


In [580]:
tmp = test_tr_clean.groupby('ID')["지점코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,주구매시간대,평균구매시간,야간구매비율,새벽구매비율,아침_구매비율,점심_구매비율,저녁_구매비율,방문지점수,주구매지점,주구매지점_이용비율
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,14,14.078571,0.000000,0.0,0.0,0.428571,0.000000,1,A202000,1.000000
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,11,15.095833,0.000000,0.0,0.0,0.250000,0.250000,2,A373000,0.750000
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,17,15.066667,0.000000,0.0,0.0,0.400000,0.600000,1,A144000,1.000000
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,18,17.884722,0.541667,0.0,0.0,0.000000,0.708333,1,A112000,1.000000
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,19,17.278889,0.466667,0.0,0.0,0.066667,0.600000,3,A112000,0.466667


In [581]:
train_ft.to_csv(f"{DATA_PATH}train_tmp.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_tmp.csv",index=False)

## 브랜드코드를 이용한 특성생성

In [582]:
train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

In [583]:
agg_list = [
    ("브랜드코드_nunique", "nunique"),
    ("선호브랜드코드", lambda x: x.mode()[0]),
    ("선호브랜드코드_구매비율", lambda x: x[x == x.mode()[0]].count() / x.count())
]

tmp = train_tr_clean.groupby("ID")["브랜드코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how="left", on="ID")
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,새벽구매비율,아침_구매비율,점심_구매비율,저녁_구매비율,방문지점수,주구매지점,주구매지점_이용비율,브랜드코드_nunique,선호브랜드코드,선호브랜드코드_구매비율
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,0.0,0.000000,0.357143,0.571429,2,A112000,0.571429,13,5405,0.142857
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,0.0,0.023810,0.380952,0.428571,3,A112000,0.738095,26,5100,0.142857
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,0.0,0.020000,0.200000,0.260000,2,A373000,0.900000,58,5159,0.120000
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,0.0,0.010811,0.248649,0.378378,3,A144000,0.805405,99,5138,0.043243
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,0.0,0.027778,0.222222,0.333333,2,A144000,0.888889,16,5100,0.472222


In [584]:
tmp = test_tr_clean.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,새벽구매비율,아침_구매비율,점심_구매비율,저녁_구매비율,방문지점수,주구매지점,주구매지점_이용비율,브랜드코드_nunique,선호브랜드코드,선호브랜드코드_구매비율
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,0.0,0.0,0.428571,0.000000,1,A202000,1.000000,5,5100,0.428571
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,0.0,0.0,0.250000,0.250000,2,A373000,0.750000,4,5149,0.250000
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,0.0,0.0,0.400000,0.600000,1,A144000,1.000000,5,5111,0.200000
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,0.0,0.0,0.000000,0.708333,1,A112000,1.000000,18,5956,0.166667
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,0.0,0.0,0.066667,0.600000,3,A112000,0.466667,12,5100,0.133333


In [585]:
train_ft.to_csv(f"{DATA_PATH}train_tmp.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_tmp.csv",index=False)

## 중분류를 이용한 특성생성

In [586]:
train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

In [587]:
agg_list = [
    ("중분류_nunique", "nunique"),
    ("주구매_중분류", lambda x: x.mode()[0]),
    
    ('주구매_중분류_아동용품', lambda x: (x.str.contains("아동|유아|신생아|완구|팬시|주니어", regex=True)).mean()),
    ('주구매_중분류_취미용품', lambda x: (x.str.contains('아웃도어|골프|스포츠|취미|수예|레포츠', regex=True)).mean()),
    ('주구매_중분류_가전제품', lambda x: (x.str.contains('TV|냉장고|취사', regex=True)).mean()),
    ('주구매_중분류_가구', lambda x: (x.str.contains('가구|식탁|쇼파|소파|침대|침구|홈', regex=True)).mean()),
    ('주구매_중분류_사치품', lambda x: (x.str.contains('보석|모피,', regex=True)).mean()),
    ("주구매_중분류_악세서리", lambda x: (x.str.contains("악세사리|액세서리|보석|핸드백|장신구|시계|ACC", regex=True)).mean()),
    ("주구매_중분류_명품", lambda x: (x.str.contains("명품|부띠끄|로얄|부틱|엘레강스", regex=True)).mean()),
    ("주구매_중분류_화장품", lambda x: (x.str.contains("화장품|향수", regex=True)).mean()),
    ("주구매_중분류_영패션", lambda x: (x.str.contains("영|캐쥬얼", regex=True)).mean()),
    ('주구매_중분류_중장년타겟제품', lambda x: (x.str.contains('건강식품|머플러|양말|도자기', regex=True)).mean()),
]

tmp = train_tr_clean.groupby("ID")["중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,주구매_중분류,주구매_중분류_아동용품,주구매_중분류_취미용품,주구매_중분류_가전제품,주구매_중분류_가구,주구매_중분류_사치품,주구매_중분류_악세서리,주구매_중분류_명품,주구매_중분류_화장품,주구매_중분류_영패션
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,골프웨어,0.142857,0.214286,0.000000,0.00,0.000000,0.000000,0.000000,0.071429,0.000000
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,영캐주얼,0.000000,0.071429,0.000000,0.00,0.000000,0.000000,0.047619,0.095238,0.190476
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,아웃도어,0.000000,0.260000,0.000000,0.01,0.000000,0.030000,0.120000,0.100000,0.100000
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,수입종합화장품,0.016216,0.054054,0.010811,0.00,0.005405,0.064865,0.000000,0.210811,0.210811
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,용기보증,0.027778,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.138889,0.027778


In [588]:
tmp = test_tr_clean.groupby('ID')["중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,주구매_중분류,주구매_중분류_아동용품,주구매_중분류_취미용품,주구매_중분류_가전제품,주구매_중분류_가구,주구매_중분류_사치품,주구매_중분류_악세서리,주구매_중분류_명품,주구매_중분류_화장품,주구매_중분류_영패션
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,용기보증,0.285714,0.000,0.0,0.0,0.0,0.000000,0.142857,0.142857,0.000000
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,남성구두,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.250000,0.000000
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,수입종합화장품,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.400000,0.200000
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,영캐주얼,0.000000,0.125,0.0,0.0,0.0,0.041667,0.000000,0.083333,0.250000
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,칼라드래디셔널,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.066667,0.266667


In [589]:
train_ft.to_csv(f"{DATA_PATH}train_tmp.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_tmp.csv",index=False)

## 대분류를 이용한 특성생성

In [590]:
train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

In [591]:
agg_list = [
    ("대분류_nunique", "nunique"),
    ("주구매_대분류", lambda x: x.mode()[0]),
    
    ("주구매_대분류_아동용품", lambda x: (x.str.contains("아동")).mean()),
    ("주구매_대분류_잡화", lambda x: (x.str.contains("영|잡화", regex=True)).mean()),
    ('주구매_대분류_스포츠', lambda x: (x.str.contains('골프|스포츠', regex=True)).mean()),
    ("주구매_대분류_가정용품", lambda x: (x.str.contains("가정용품")).mean()),
    ("주구매_대분류_명품", lambda x: (x.str.contains("로얄|명품", regex=True)).mean()),
    ("주구매_대분류_생식품", lambda x: (x.str.contains("생식품")).mean()),
]

tmp = train_tr_clean.groupby("ID")["대분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,주구매_중분류_화장품,주구매_중분류_영패션,대분류_nunique,주구매_대분류,주구매_대분류_아동용품,주구매_대분류_잡화,주구매_대분류_스포츠,주구매_대분류_가정용품,주구매_대분류_명품,주구매_대분류_생식품
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,0.071429,0.000000,9,패션잡화,0.142857,0.285714,0.214286,0.285714,0.000000,0.071429
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,0.095238,0.190476,12,영플라자,0.023810,0.595238,0.071429,0.000000,0.047619,0.023810
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,0.100000,0.100000,14,명품잡화,0.160000,0.360000,0.420000,0.030000,0.260000,0.030000
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,0.210811,0.210811,16,케주얼_구두_아동,0.286486,0.378378,0.113514,0.037838,0.021622,0.021622
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,0.138889,0.027778,8,공산품파트,0.111111,0.250000,0.000000,0.000000,0.194444,0.194444


In [592]:
tmp = test_tr_clean.groupby('ID')["대분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,주구매_중분류_화장품,주구매_중분류_영패션,대분류_nunique,주구매_대분류,주구매_대분류_아동용품,주구매_대분류_잡화,주구매_대분류_스포츠,주구매_대분류_가정용품,주구매_대분류_명품,주구매_대분류_생식품
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,0.142857,0.000000,5,공산품,0.285714,0.142857,0.000000,0.000000,0.0,0.142857
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,0.250000,0.000000,3,명품잡화,0.000000,0.500000,0.000000,0.000000,0.5,0.000000
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,0.400000,0.200000,3,잡화파트,0.400000,0.400000,0.000000,0.000000,0.0,0.000000
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,0.083333,0.250000,4,영플라자,0.000000,0.916667,0.041667,0.041667,0.0,0.000000
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,0.066667,0.266667,6,여성캐주얼,0.200000,0.133333,0.066667,0.000000,0.0,0.000000


In [593]:
train_ft.to_csv(f"{DATA_PATH}train_tmp.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_tmp.csv",index=False)

## 구매가격을 이용한 특성생성

In [594]:
train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

- 구매가격 사분위수

In [595]:
high_purchase = train_tr_clean["구매가격"].quantile(0.75)
low_purchase = train_tr_clean["구매가격"].quantile(0.25)

high_purchase, low_purchase

(120000.0, 26334.0)

- 환불가격 사분위수

In [596]:
low_refund, high_refund = refund_train["구매가격"].quantile(0.75), refund_train["구매가격"].quantile(0.25)
low_refund, high_refund

(-55000.0, -239000.0)

- 순수 구매 관련 컬럼

In [597]:
# 중가제품 구매비율, 구매금액 변동성 추가
# 고가제품 구매건수 삭제

agg_list = [
    ("총구매금액", "sum"),
    ("구매건수", "count"),
    ("평균구매금액", "mean"),
    ("구매금액_median", 'median'),
    ("최대구매금액", 'max'),
    ("최소구매금액", 'min'),
    ("구매금액표준편차", 'std'),
    ("구매금액_skew", 'skew'),
    ("구매금액_kurtosis", lambda x: x.kurtosis()),
    ("고가제품_구매비율", lambda x: np.mean(x > high_purchase)),
    ("중가제품_구매비율", lambda x: np.mean((x >= low_purchase) & (x <= high_purchase))),
    ("저가제품_구매비율", lambda x: np.mean(x < low_purchase)),
    ("구매금액_변동성", lambda x: x.std() / x.mean() if x.mean() != 0 else 0),
]

tmp = train_tr_clean.groupby("ID")["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, "left", "ID")
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,구매금액_median,최대구매금액,최소구매금액,구매금액표준편차,구매금액_skew,구매금액_kurtosis,고가제품_구매비율,중가제품_구매비율,저가제품_구매비율,구매금액_변동성
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,187000.0,1000000,20000,313025.189100,1.670489,1.990929,0.714286,0.214286,0.071429,1.043119
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,90450.0,698000,5600,126592.626144,2.651821,9.771083,0.333333,0.547619,0.119048,1.054306
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,90500.0,2770000,4400,419490.327837,4.346572,21.461791,0.430000,0.430000,0.140000,1.895112
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,120000.0,1492000,-155000,240415.848887,2.602747,8.168448,0.491892,0.421622,0.086486,1.249536
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,41210.5,900000,7640,179263.263965,3.017610,10.550421,0.194444,0.472222,0.333333,1.617873


In [598]:
tmp = test_tr_clean.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, 'left', "ID")
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,구매금액_median,최대구매금액,최소구매금액,구매금액표준편차,구매금액_skew,구매금액_kurtosis,고가제품_구매비율,중가제품_구매비율,저가제품_구매비율,구매금액_변동성
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,58536.0,110000,26643,32293.148313,0.702551,-1.103688,0.000000,1.000000,0.000000,0.526218
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,67500.0,118000,37600,33658.431336,0.856705,1.438447,0.000000,1.000000,0.000000,0.463296
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,95000.0,326000,38000,117233.954126,1.969409,4.075918,0.200000,0.800000,0.000000,0.965683
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,55250.0,403000,7040,102440.792115,1.507265,2.087113,0.291667,0.291667,0.416667,1.066286
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,79000.0,376000,16200,99879.156875,1.839077,3.277987,0.266667,0.600000,0.133333,0.955056


- 환불된 데이터에 대한 컬럼

In [599]:
# 중가제품환불비율 추가
# 총환불금액, 평균환불금액 추가

agg_list = [
    ("환불건수", "count"),
    ("총환불금액", "sum"),
    ("평균환불금액", "mean"),
    ("최소환불금액", "max"),
    ("최대환불금액", "min"),
    ("환불금액표준편차", "std"),
    ("환불금액_skew", "skew"), 
    ("환불금액_kurtosis", lambda x: x.kurtosis()),
    ("고가제품환불비율", lambda x: np.mean(x < high_refund)),
    ("중가제품환불비율", lambda x: np.mean((x >= high_refund) & (x <= low_refund))),
    ("저가제품환불비율", lambda x: np.mean(x > low_refund)),
]

tmp = refund_train.groupby("ID")["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, "left", "ID").fillna(0)
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,총환불금액,평균환불금액,최소환불금액,최대환불금액,환불금액표준편차,환불금액_skew,환불금액_kurtosis,고가제품환불비율,중가제품환불비율,저가제품환불비율
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,-2517000.0,-839000.000000,-205000.0,-1236000.0,554857.639400,1.571433,0.000000,0.666667,0.333333,0.000000
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,-1072500.0,-153214.285714,-4500.0,-288000.0,110664.604829,0.310886,-1.234108,0.285714,0.428571,0.285714
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,-2867800.0,-204842.857143,-58000.0,-768000.0,194802.324715,-2.219921,5.204875,0.214286,0.785714,0.000000
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,-6954400.0,-534953.846154,-49000.0,-1204000.0,379184.767221,-0.381901,-1.126703,0.692308,0.230769,0.076923


In [600]:
tmp = refund_test.groupby("ID")["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, "left", "ID").fillna(0)
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,총환불금액,평균환불금액,최소환불금액,최대환불금액,환불금액표준편차,환불금액_skew,환불금액_kurtosis,고가제품환불비율,중가제품환불비율,저가제품환불비율
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,-1092000.0,-273000.000000,-140000.0,-403000.0,137663.841779,0.014171,-5.642846,0.5,0.500000,0.000000
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,-372200.0,-124066.666667,-38000.0,-196200.0,80015.081912,0.759841,0.000000,0.0,0.666667,0.333333


In [624]:
refund_pairs_train

,ID,구매가격_refund,지점코드,대분류,중분류,브랜드코드,index_refund,구매날짜_refund,구매시각_refund,index_purchase,구매가격_purchase,구매날짜_purchase,구매시각_purchase
0,train_0,-1236000,A144000,남성정장스포츠,골프웨어,5452,304854,2004-11-28,15:20:00,299821,1236000,2004-11-23,13:10:00
1,train_0,-1076000,A144000,남성정장스포츠,골프웨어,5452,321254,2004-12-09,12:30:00,304855,1076000,2004-11-28,15:20:00
2,train_0,-205000,A144000,남성정장스포츠,골프웨어,5409,321257,2004-12-09,12:30:00,299858,205000,2004-11-23,13:30:00
3,train_10,-139000,A144000,남성정장스포츠,셔츠,5440,210764,2004-09-24,13:00:00,202034,139000,2004-09-17,13:20:00
4,train_10,-30000,A144000,잡화파트,수입종합화장품,5112,397605,2005-02-04,13:20:00,397583,30000,2005-02-04,13:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33625,train_9997,-19000,A112000,영플라자,영캐주얼,5273,428694,2005-03-01,15:10:00,398672,19000,2005-02-04,18:50:00
33626,train_9999,-228000,A112000,가정용품,주방용품,5514,187543,2004-09-04,15:33:00,157153,228000,2004-08-13,14:53:00
33627,train_9999,-176000,A112000,가정용품,주방용품,6120,157143,2004-08-13,14:50:00,157152,176000,2004-08-13,14:53:00
33628,train_9999,-85000,A112000,영플라자,스포츠캐주얼,5169,419720,2005-02-23,13:43:00,415368,85000,2005-02-18,19:43:00


### 구매가격을 이용한 등급 특성 생성

In [601]:
def categorize_customer(total_amount):
    if total_amount >= 20_000_000:
        return 'Diamond'
    elif total_amount >= 15_000_000:
        return 'Platinum'
    elif total_amount >= 10_000_000:
        return 'Gold'
    elif total_amount >= 6_000_000:
        return 'Black'
    else:
        return 'Normal'
    
tmp = train_tr_clean.groupby('ID')['구매가격'].sum().reset_index()
train_ft['고객등급'] = tmp['구매가격'].apply(categorize_customer)
train_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,평균환불금액,최소환불금액,최대환불금액,환불금액표준편차,환불금액_skew,환불금액_kurtosis,고가제품환불비율,중가제품환불비율,저가제품환불비율,고객등급
0,train_0,9,2004-05-07,2005-01-16,254,28,2,1.555556,22.744789,104,...,-839000.000000,-205000.0,-1236000.0,554857.639400,1.571433,0.000000,0.666667,0.333333,0.000000,일반
1,train_1,21,2004-05-11,2005-04-27,351,16,1,2.000000,11.739134,3,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,일반
2,train_2,55,2004-05-06,2005-04-26,355,6,11,1.818182,5.045270,4,...,-153214.285714,-4500.0,-288000.0,110664.604829,0.310886,-1.234108,0.285714,0.428571,0.285714,VIP
3,train_3,90,2004-05-11,2005-04-20,344,3,19,2.055556,3.137146,10,...,-204842.857143,-58000.0,-768000.0,194802.324715,-2.219921,5.204875,0.214286,0.785714,0.000000,일반
4,train_4,25,2004-05-09,2005-03-17,312,12,6,1.440000,11.902872,44,...,-534953.846154,-49000.0,-1204000.0,379184.767221,-0.381901,-1.126703,0.692308,0.230769,0.076923,일반


In [602]:
tmp = test_tr_clean.groupby('ID')['구매가격'].sum().reset_index()
test_ft['고객등급'] = tmp['구매가격'].apply(categorize_customer)
test_ft.head()

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,평균환불금액,최소환불금액,최대환불금액,환불금액표준편차,환불금액_skew,환불금액_kurtosis,고가제품환불비율,중가제품환불비율,저가제품환불비율,고객등급
0,test_0,5,2004-05-16,2004-10-26,163,32,3,1.400000,30.416096,186,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,일반
1,test_1,3,2004-09-11,2005-02-23,165,55,1,1.333333,57.879185,66,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,일반
2,test_2,2,2004-08-16,2004-09-02,17,8,0,2.500000,7.361216,240,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,일반
3,test_3,14,2004-07-22,2005-04-19,271,19,1,1.714286,17.715031,11,...,-273000.000000,-140000.0,-403000.0,137663.841779,0.014171,-5.642846,0.5,0.500000,0.000000,일반
4,test_4,7,2004-05-25,2005-02-13,264,37,3,2.142857,36.517818,76,...,-124066.666667,-38000.0,-196200.0,80015.081912,0.759841,0.000000,0.0,0.666667,0.333333,일반


In [603]:
train_ft.to_csv(f"{DATA_PATH}train_tmp.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_tmp.csv",index=False)

## pivot_table을 이용한 특성 생성

In [604]:
train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

### ID, 지점코드별 구매횟수

In [605]:
train_tmp = pd.pivot_table(
    train_tr_clean,
    index="ID",
    columns="지점코드",
    values="구매가격",
    aggfunc="count",
    fill_value=0,
).add_prefix("pivot_지점코드_")

train_tmp

지점코드,pivot_지점코드_A112000,pivot_지점코드_A144000,pivot_지점코드_A202000,pivot_지점코드_A373000
ID,,,,
train_0,8,6,0,0
train_1,31,2,0,9
train_10,0,118,10,0
train_100,5,9,0,6
train_1000,0,2,0,13
...,...,...,...,...
train_9995,1,0,0,0
train_9996,1,22,0,0
train_9997,15,0,0,0


In [606]:
test_tmp = pd.pivot_table(
    test_tr_clean,
    index="ID",
    columns="지점코드",
    values="구매가격",
    aggfunc="count",
    fill_value=0,
).add_prefix("pivot_지점코드_")

test_tmp

지점코드,pivot_지점코드_A112000,pivot_지점코드_A144000,pivot_지점코드_A202000,pivot_지점코드_A373000
ID,,,,
test_0,0,0,7,0
test_1,1,0,0,3
test_10,28,0,0,0
test_100,103,3,0,5
test_1000,0,0,0,3
...,...,...,...,...
test_9995,16,0,0,0
test_9996,0,0,44,0
test_9997,0,0,0,21


### ID, 중분류별 구매횟수

In [607]:
train_tmp = train_tmp.merge(
    pd.pivot_table(
        train_tr_clean,
        index="ID",
        columns="중분류",
        values="구매가격",
        aggfunc="count",
        fill_value=0,
    )
    .add_prefix("pivot_중분류_")
    .reset_index(),
    on="ID",
    how="left",
)

train_tmp

,ID,pivot_지점코드_A112000,pivot_지점코드_A144000,pivot_지점코드_A202000,pivot_지점코드_A373000,pivot_중분류_DC캐주얼,pivot_중분류_GBR 지원,pivot_중분류_L_B침구,pivot_중분류_NB제화,pivot_중분류_NB핸드백,...,pivot_중분류_행사슈즈,pivot_중분류_행사핸드백,pivot_중분류_향수,pivot_중분류_헤어ACC,pivot_중분류_헤어악세사리,pivot_중분류_헤어액세사리,pivot_중분류_홈데코,pivot_중분류_화장잡화,pivot_중분류_화장품,pivot_중분류_훼미닌부틱
0,train_0,8,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,train_1,31,2,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,train_10,0,118,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,train_100,5,9,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,train_1000,0,2,0,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_9995,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14936,train_9996,1,22,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14937,train_9997,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14938,train_9998,24,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [608]:
test_tmp = test_tmp.merge(
    pd.pivot_table(
        test_tr_clean,
        index="ID",
        columns="중분류",
        values="구매가격",
        aggfunc="count",
        fill_value=0,
    )
    .add_prefix("pivot_중분류_")
    .reset_index(),
    on="ID",
    how="left",
)

test_tmp

,ID,pivot_지점코드_A112000,pivot_지점코드_A144000,pivot_지점코드_A202000,pivot_지점코드_A373000,pivot_중분류_DC캐주얼,pivot_중분류_GBR 지원,pivot_중분류_L_B침구,pivot_중분류_NB제화,pivot_중분류_NB핸드백,...,pivot_중분류_행사슈즈,pivot_중분류_행사핸드백,pivot_중분류_향수,pivot_중분류_헤어ACC,pivot_중분류_헤어악세사리,pivot_중분류_헤어액세사리,pivot_중분류_홈데코,pivot_중분류_화장잡화,pivot_중분류_화장품,pivot_중분류_훼미닌부틱
0,test_0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test_1,1,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test_10,28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test_100,103,3,0,5,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,test_1000,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_9995,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
12221,test_9996,0,0,44,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
12222,test_9997,0,0,0,21,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
12223,test_9998,0,0,62,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### ID, 대분류별 구매횟수

In [609]:
train_tmp = train_tmp.merge(
    pd.pivot_table(
        train_tr_clean,
        index="ID",
        columns="대분류",
        values="구매가격",
        aggfunc="count",
        fill_value=0,
    )
    .add_prefix("pivot_대분류_")
    .reset_index(),
    on="ID",
    how="left",
)

train_tmp

,ID,pivot_지점코드_A112000,pivot_지점코드_A144000,pivot_지점코드_A202000,pivot_지점코드_A373000,pivot_중분류_DC캐주얼,pivot_중분류_GBR 지원,pivot_중분류_L_B침구,pivot_중분류_NB제화,pivot_중분류_NB핸드백,...,pivot_대분류_여성캐주얼,pivot_대분류_여성캐쥬얼,pivot_대분류_영라이브,pivot_대분류_영어덜트캐쥬얼,pivot_대분류_영캐릭터,pivot_대분류_영플라자,pivot_대분류_잡화,pivot_대분류_잡화파트,pivot_대분류_케주얼_구두_아동,pivot_대분류_패션잡화
0,train_0,8,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,3
1,train_1,31,2,0,9,0,0,0,0,0,...,6,0,0,0,4,11,0,0,1,8
2,train_10,0,118,10,0,0,0,0,0,0,...,0,0,0,0,0,0,2,19,5,0
3,train_100,5,9,0,6,0,0,0,0,0,...,0,0,0,0,0,1,0,3,4,1
4,train_1000,0,2,0,13,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_9995,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14936,train_9996,1,22,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,2,12,0
14937,train_9997,15,0,0,0,0,0,0,0,0,...,2,0,0,0,2,7,0,0,0,1
14938,train_9998,24,0,0,5,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,9


In [610]:
test_tmp = test_tmp.merge(
    pd.pivot_table(
        test_tr_clean,
        index="ID",
        columns="대분류",
        values="구매가격",
        aggfunc="count",
        fill_value=0,
    )
    .add_prefix("pivot_대분류_")
    .reset_index(),
    on="ID",
    how="left",
)

test_tmp

,ID,pivot_지점코드_A112000,pivot_지점코드_A144000,pivot_지점코드_A202000,pivot_지점코드_A373000,pivot_중분류_DC캐주얼,pivot_중분류_GBR 지원,pivot_중분류_L_B침구,pivot_중분류_NB제화,pivot_중분류_NB핸드백,...,pivot_대분류_여성캐주얼,pivot_대분류_여성캐쥬얼,pivot_대분류_영라이브,pivot_대분류_영어덜트캐쥬얼,pivot_대분류_영캐릭터,pivot_대분류_영플라자,pivot_대분류_잡화,pivot_대분류_잡화파트,pivot_대분류_케주얼_구두_아동,pivot_대분류_패션잡화
0,test_0,0,0,7,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,test_1,1,0,0,3,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,test_10,28,0,0,0,0,0,0,0,0,...,0,0,0,0,4,3,0,0,0,11
3,test_100,103,3,0,5,0,0,0,0,1,...,5,0,0,0,1,1,0,2,0,16
4,test_1000,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_9995,16,0,0,0,0,0,0,0,0,...,2,0,0,0,3,3,0,0,0,2
12221,test_9996,0,0,44,0,0,0,0,0,0,...,0,2,4,4,0,0,4,0,0,0
12222,test_9997,0,0,0,21,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
12223,test_9998,0,0,62,0,0,0,0,0,0,...,0,6,10,1,0,0,8,0,0,0


### ID, 브랜드코드별 구매횟수

In [611]:
train_tmp = train_tmp.merge(
    pd.pivot_table(
        train_tr_clean,
        index="ID",
        columns="브랜드코드",
        values="구매가격",
        aggfunc="count",
        fill_value=0,
    )
    .add_prefix("pivot_브랜드코드_")
    .reset_index(),
    on="ID",
    how='left'    
)

train_tmp

,ID,pivot_지점코드_A112000,pivot_지점코드_A144000,pivot_지점코드_A202000,pivot_지점코드_A373000,pivot_중분류_DC캐주얼,pivot_중분류_GBR 지원,pivot_중분류_L_B침구,pivot_중분류_NB제화,pivot_중분류_NB핸드백,...,pivot_브랜드코드_6984,pivot_브랜드코드_6985,pivot_브랜드코드_6987,pivot_브랜드코드_6988,pivot_브랜드코드_6989,pivot_브랜드코드_6990,pivot_브랜드코드_6992,pivot_브랜드코드_6994,pivot_브랜드코드_6995,pivot_브랜드코드_6996
0,train_0,8,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,train_1,31,2,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,train_10,0,118,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,train_100,5,9,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,train_1000,0,2,0,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_9995,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14936,train_9996,1,22,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14937,train_9997,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14938,train_9998,24,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [612]:
test_tmp = test_tmp.merge(
    pd.pivot_table(
        test_tr_clean,
        index="ID",
        columns="브랜드코드",
        values="구매가격",
        aggfunc="count",
        fill_value=0,
    )
    .add_prefix("pivot_브랜드코드_")
    .reset_index(),
    on="ID",
    how='left'    
)

test_tmp

,ID,pivot_지점코드_A112000,pivot_지점코드_A144000,pivot_지점코드_A202000,pivot_지점코드_A373000,pivot_중분류_DC캐주얼,pivot_중분류_GBR 지원,pivot_중분류_L_B침구,pivot_중분류_NB제화,pivot_중분류_NB핸드백,...,pivot_브랜드코드_6985,pivot_브랜드코드_6986,pivot_브랜드코드_6987,pivot_브랜드코드_6988,pivot_브랜드코드_6990,pivot_브랜드코드_6992,pivot_브랜드코드_6994,pivot_브랜드코드_6995,pivot_브랜드코드_6996,pivot_브랜드코드_6998
0,test_0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test_1,1,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test_10,28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test_100,103,3,0,5,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,test_1000,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_9995,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12221,test_9996,0,0,44,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12222,test_9997,0,0,0,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12223,test_9998,0,0,62,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [626]:
요일별_구매금액_sum = pd.pivot_table(
    train_tr_clean,
    index='ID',
    columns=train_tr_clean['구매날짜'].dt.day_name(),
    values='구매가격',
    aggfunc='sum',
    fill_value=0
)

요일별_구매금액_sum.columns = [f"{day}_sum" for day in 요일별_구매금액_sum.columns]
train_tmp = train_tmp.merge(요일별_구매금액_sum, how='left', on='ID')
train_tmp.head()

ID
train_0        4201200
train_1        5043025
train_10      11234870
train_100      1875887
train_1000      757500
                ...   
train_9995       36000
train_9996     4774410
train_9997      795700
train_9998     2069325
train_9999     6970530
Length: 14940, dtype: int64

In [ ]:
요일별_구매금액_sum = pd.pivot_table(
    test_tr_clean,
    index='ID',
    columns=test_tr_clean['구매날짜'].dt.day_name(),
    values='구매가격',
    aggfunc='sum',
    fill_value=0
)

요일별_구매금액_sum.columns = [f"{day}_sum" for day in 요일별_구매금액_sum.columns]
test_tmp = test_tmp.merge(요일별_구매금액_sum, how='left', on='ID')
test_tmp.head()

### 최종 pivot table merge

In [613]:
train_ft = train_ft.merge(train_tmp, on="ID", how="left")

for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]
test_ft = test_ft.merge(test_tmp, how="left", on="ID")

In [614]:
train_ft.to_csv(f"{DATA_PATH}train_tmp.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_tmp.csv",index=False)

## 추가 피처 생성

In [615]:
train_ft = pd.read_csv(f"{DATA_PATH}train_tmp.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_tmp.csv")

In [616]:
weekdays = ["월요일_구매비율", "화요일_구매비율", "수요일_구매비율", "목요일_구매비율", "금요일_구매비율"]

train_ft["평일_구매비율"] = train_ft[weekdays].sum(axis=1)
test_ft["평일_구매비율"] = test_ft[weekdays].sum(axis=1)

In [617]:
weekend = ["토요일_구매비율", "일요일_구매비율"]

train_ft["주말_구매비율"] = train_ft[weekend].sum(axis=1)
test_ft["주말_구매비율"] = test_ft[weekend].sum(axis=1)

In [618]:
# 추가

train_ft["주말_방문_선호도"] = train_ft["주말_구매비율"] / (train_ft["평일_구매비율"] + 1)
train_ft["방문일수_대비_구매건수"] = train_ft["구매건수"] / train_ft["총방문일수"]
train_ft["전체_성수기_구매비율"] = train_ft[["추석성수기_구매비율", "연말연시_구매비율", "설날성수기_구매비율"]].sum(axis=1)
train_ft["전체_준성수기_구매비율"] = train_ft[["여름시즌_구매비율", "신학기_구매비율"]].sum(axis=1)
train_ft["전체_중간기_구매비율"] = train_ft[["여름끝추석전_구매비율", "연말연시끝설날전_구매비율", "신학기후_구매비율"]].sum(axis=1)
train_ft["전체_비수기_구매비율"] = train_ft[["초여름_구매비율", "추석후연말전_구매비율", "봄철비수기_구매비율"]].sum(axis=1)
train_ft["성수기_구매집중도"] = train_ft["전체_성수기_구매비율"] / (train_ft["전체_비수기_구매비율"] + 1)
train_ft["계절_구매변동성"] = train_ft[["봄_구매비율", "여름_구매비율", "가을_구매비율", "겨울_구매비율"]].std(axis=1)

test_ft["주말_방문_선호도"] = test_ft["주말_구매비율"] / (test_ft["평일_구매비율"] + 1)
test_ft["방문일수_대비_구매건수"] = test_ft["구매건수"] / test_ft["총방문일수"]
test_ft["전체_성수기_구매비율"] = test_ft[["추석성수기_구매비율", "연말연시_구매비율", "설날성수기_구매비율"]].sum(axis=1)
test_ft["전체_준성수기_구매비율"] = test_ft[["여름시즌_구매비율", "신학기_구매비율"]].sum(axis=1)
test_ft["전체_중간기_구매비율"] = test_ft[["여름끝추석전_구매비율", "연말연시끝설날전_구매비율", "신학기후_구매비율"]].sum(axis=1)
test_ft["전체_비수기_구매비율"] = test_ft[["초여름_구매비율", "추석후연말전_구매비율", "봄철비수기_구매비율"]].sum(axis=1)
test_ft["성수기_구매집중도"] = test_ft["전체_성수기_구매비율"] / (test_ft["전체_비수기_구매비율"] + 1)
test_ft["계절_구매변동성"] = test_ft[["봄_구매비율", "여름_구매비율", "가을_구매비율", "겨울_구매비율"]].std(axis=1)

In [619]:
# 삭제

# train_ft["최종구매건수"] = train_ft["구매건수"] - train_ft["환불건수"]
# train_ft["총환불금액"] = train_ft["총사용금액"] - train_ft["총사용금액"]
# train_ft["평균구매금액"] = train_ft["총사용금액"] / train_ft["최종구매건수"]
# train_ft["충동구매비율"] = train_ft["고액구매건수"] / train_ft["최종구매건수"]

# test_ft["최종구매건수"] = test_ft["구매건수"] - test_ft["환불건수"]
# test_ft["총환불금액"] = test_ft["총사용금액"] - test_ft["총사용금액"]
# test_ft["평균구매금액"] = test_ft["총사용금액"] / test_ft["최종구매건수"]
# test_ft["충동구매비율"] = test_ft["고액구매건수"] / test_ft["최종구매건수"]

In [620]:
# 추가

train_ft["구매주기_대비_구매금액"] = train_ft["총구매금액"] / train_ft["구매주기"]
train_ft["방문당_평균구매금액"] = train_ft["총구매금액"] / train_ft["총방문일수"]
train_ft["구매금액_대비_환불금액"] = train_ft["총환불금액"] / train_ft["총구매금액"]
train_ft["환불건수_대비_구매건수"] = train_ft["환불건수"] / train_ft["구매건수"]

test_ft["구매주기_대비_구매금액"] = test_ft["총구매금액"] / test_ft["구매주기"]
test_ft["방문당_평균구매금액"] = test_ft["총구매금액"] / test_ft["총방문일수"]
test_ft["구매금액_대비_환불금액"] = abs(test_ft["총환불금액"]) / test_ft["총구매금액"]
test_ft["환불건수_대비_구매건수"] = test_ft["환불건수"] / test_ft["구매건수"]

# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [621]:
train_ft.shape, test_ft.shape

((14940, 2299), (12225, 2299))

# 추출한 피처 저장하기

In [622]:
train_ft.to_csv(f"{DATA_PATH}train_common.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_common.csv",index=False)